In [3]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import altair as alt
import plotly.express as px
import plotly.graph_objects as go

In [4]:
team_id = 251
team_color = "#bf5700"
season = 2023
url = f"http://www.espn.com/womens-college-basketball/team/schedule/_/id/{team_id}/season/{season}"


# Pandas Method

In [5]:
# table = pd.read_html(url)[0]
# table.head()

# BS Method

In [6]:
page = rq.get(url)

In [7]:
soup = bs(page.content)

In [8]:
table = soup.table
data = {'Opponent': [],
        'Attendance': [],
        'Location': []} 
for row in table.find_all('tr')[2:]:
    row_cells = row.find_all('td')
    try:
        opponent = row_cells[1].find_all('a')[1].contents[0]
        game_link = row_cells[2].find('a')['href']
        if(row_cells[1].find_all('span')[2].contents[-1] == "*"):
            location = "Neutral"
        else:
            location = "Home" if row_cells[1].find_all('span')[0].contents[0] == "vs" else "Away"
    except:
        break
    # print(game_link)
    game_page = rq.get(game_link)
    game_soup = bs(game_page.content)
    atndnce = game_soup.find('div', {"class":"Attendance__Numbers"})
    atndnce = atndnce.contents[4] if atndnce != None else 0
    atndnce = ''.join(filter(str.isdigit, atndnce)) if atndnce != 0 else atndnce
    # add current row data to dictionary
    if atndnce != 0:
        data["Opponent"].append(opponent)
        data["Attendance"].append(int(atndnce))
        data["Location"].append(location)

df= pd.DataFrame(data)

In [9]:
df

,Opponent,Attendance,Location
0,Louisiana,5658,Home
1,UConn,10167,Away
2,Marquette,376,Neutral
3,Louisville,365,Neutral
4,Rutgers,155,Neutral
5,Princeton,5137,Home
6,South Florida,5021,Home
7,Southern,4987,Home
8,Alabama State,5116,Home
9,Jackson State,1200,Neutral


Altair

In [13]:
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Opponent:O',sort = data["Opponent"], title=None),
    y='Attendance',
    column='Location:N',
    color=alt.Color('Location',
                    scale=alt.Scale(
                        domain=['Home', 'Away' ,'Neutral'],
                        range=[team_color, 'darkgray', 'lightgray']))
).resolve_scale(
    x='independent',
    # column="Location:N"
).properties(
    title="TX"
)
chart

/opt/homebrew/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



alt.Chart(...)

In [ ]:
print(chart.to_json())

Plotly

In [12]:
fig = go.Figure()
fig.add_bar(x=df.Opponent, y=df.Attendance)


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
df = pd.DataFrame({'x': ['Alpha', 'Beta', 'Beta', 'Beta',
                  'Gamma', 'Delta', 'Delta'], 'y': [12, 11, 9, 7, 5, 3, 1]})

fig = px.bar(df, x=df.index, y='y')
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=df.index,
        ticktext=df.x
    )
)
fig.show()